## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_maps_api

# Configure gmaps API key
gmaps.configure(api_key=google_maps_api)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Country
0,0,Punta Arenas,-53.1500,-70.9167,57.31,41,0,17.27,clear sky,CL
1,1,Sinazongwe,-17.2614,27.4618,75.65,82,100,5.10,overcast clouds,ZM
2,2,Hermanus,-34.4187,19.2345,64.80,62,0,19.35,clear sky,ZA
3,3,Cape Town,-33.9258,18.4232,72.12,62,0,8.01,clear sky,ZA
4,4,Avarua,-21.2078,-159.7750,78.85,89,75,9.22,light rain,CK


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_filt_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
city_data_filt_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
Country                2
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_data_filt_clean_df = city_data_filt_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_filt_clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Sinazongwe,ZM,75.65,overcast clouds,-17.2614,27.4618,
3,Cape Town,ZA,72.12,clear sky,-33.9258,18.4232,
4,Avarua,CK,78.85,light rain,-21.2078,-159.7750,
8,Hithadhoo,MV,81.99,few clouds,-0.6000,73.0833,
13,San Cristobal,VE,77.18,broken clouds,7.7669,-72.2250,
17,Hilo,US,75.61,overcast clouds,19.7297,-155.0900,
27,Tacna,PE,71.80,clear sky,-18.0056,-70.2483,
29,Puerto Ayora,EC,82.36,broken clouds,-0.7393,-90.3518,
31,Rikitea,PF,78.46,clear sky,-23.1203,-134.9692,
34,Butaritari,KI,82.51,few clouds,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_maps_api
}

# 6b. Iterate through the hotel DataFrame.
for row, column in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = column["Lat"]
    lng = column["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[row,"Hotel Name"] = response["results"][0]["name"]
    except(KeyError, IndexError):
        print("Skipping City")            
        

Skipping City
Skipping City
Skipping City
Skipping City
Skipping City
Skipping City
Skipping City
Skipping City
Skipping City
Skipping City
Skipping City
Skipping City
Skipping City
Skipping City
Skipping City
Skipping City


In [11]:
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
40,San Lazaro,BZ,83.98,scattered clouds,18.0387,-88.6622,
61,Merauke,ID,80.62,overcast clouds,-8.4667,140.3333,
100,Gao,ML,71.38,clear sky,16.6362,1.6370,
101,Panguna,PG,70.54,overcast clouds,-6.3164,155.4848,
206,Thinadhoo,MV,81.91,overcast clouds,0.5333,72.9333,
209,Adrar,MR,71.98,few clouds,20.5022,-10.0711,
238,Balaipungut,ID,73.81,moderate rain,1.0500,101.2833,
302,Bongandanga,CD,76.96,few clouds,1.5000,21.0500,
445,Birao,CF,81.27,few clouds,10.2849,22.7882,
448,Bafata,GW,89.53,clear sky,12.1667,-14.7500,


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}, Temp: {Max Temp}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))